## INTRODUZIONE AL PROGETTO ED OBIETTIVI:

In questo progetto esploriamo l'applicazione dell'informatica quantistica al problema della classificazione supervisionata, attraverso l'implementazione di un algoritmo variazionale ibrido. Utilizzando un approccio VQC (Variational Quantum Classifier), combiniamo circuiti quantistici parametrizzati con ottimizzatori classici per apprendere decision boundaries in uno spazio di feature trasformato quantisticamente.

L’obiettivo principale è confrontare le performance del classificatore al variare di diverse componenti, tra cui:

* Tecniche di preprocessing del dataset 
* Tipologia di ansatz quantistico 
* Scelte di encoding del dato classico 
* Ottimizzatori classici 
* Presenza o assenza di rumore quantistico

Il progetto si sviluppa prevalentemente in ambiente simulato, sfruttando strumenti come Qiskit per la costruzione e l’esecuzione dei circuiti. Particolare attenzione è data anche all’interpretabilità dei risultati grazie ad un'analisi sistematica attraverso la quale intendiamo valutare i vantaggi e le criticità dei classificatori quantistici in scenari realistici, contribuendo alla comprensione della loro efficacia in confronto con i modelli classici.

P.s. I markdown successivi hanno il solo scopo di dare una breve indicazione del funzionamento del singolo blocco di codice, per le motivazioni associate alle scelte progettuali e la valutazione dei risultati si prega di consultare la relazione.

 
 ## CODICE

Questa sezione assicura la disponibilità di tutti i pacchetti fondamentali
per l'esecuzione del progetto. In particolare:
 - sklearn: per la classificazione supervisionata e la validazione dei modelli.
 - pandas: per la gestione e manipolazione dei dataset.
 - seaborn: per la visualizzazione grafica dei risultati e delle distribuzioni.
 - qiskit: framework principale per la programmazione quantistica.
 - qiskit-algorithms: moduli avanzati di Qiskit per l'uso di algoritmi ibridi.
 - pylatexenc: supporto per la visualizzazione di output in LaTeX (es. circuiti).

In [ ]:
# Installazione delle librerie necessarie
!pip install sklearn
!pip install pandas
!pip install seaborn
!pip install qiskit
!pip install pylatexenc

Importiamo le librerie necessarie per lo sviluppo del progetto

In [ ]:
#IMPORT NECESSARI

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, confusion_matrix, classification_report, log_loss)
from scipy.optimize import minimize
from sklearn.feature_selection import SelectKBest, f_classif

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, PauliFeatureMap
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit.primitives import StatevectorSampler

Il presente blocco di codice costituisce la funzione principale (main) del nostro progetto, il flusso operativo si articola in quattro fasi principali:

1. __Caricamento e preprocessing del dataset:__ viene utilizzato il dataset Wine della libreria sklearn, i cui attributi sono sottoposti a standardizzazione, selezione delle feature (NEL BLOCCO SOTTOSTANTE SONO PRESENTI COMMENTI RELATIVI AL CODICE USATO PER EFFETTUARE LE VARIE TIPOLOGIE DI FEATURES SELECTION) e normalizzazione, al fine di renderli compatibili con l'elaborazione quantistica.

2. __Esecuzione degli esperimenti quantistici:__ impiegando un numero di qubit pari al numero di feature selezionate, vengono eseguiti esperimenti di classificazione quantistica mediante circuiti quantistici simulati, allo scopo di valutare le performance dei modelli quantistici su dati reali pre-elaborati.

3. __Analisi dei risultati:__ infine, viene effettuata un’analisi dettagliata delle performance ottenute in termini di accuratezza e altre metriche significative.

Il codice è progettato per offrire una pipeline completa e modulare, utile per esplorare l’applicabilità del Quantum Machine Learning a problemi di classificazione multiclasse reali.

In [ ]:
# =============================================================================
# MAIN COMPLETO PER QUANTUM WINE CLASSIFICATION
# =============================================================================

# Impostazioni per i plot
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def main():
    """
    Funzione main che esegue tutto il pipeline di classificazione quantistica
    """

    # =============================================================================
    # 1. CARICAMENTO E PREPROCESSING DEL DATASET WINE CON FEATURE SELECTION
    # =============================================================================

    print("=" * 80)
    print("CARICAMENTO DATASET WINE E PREPROCESSING CON FEATURE SELECTION")
    print("=" * 80)

    # Caricamento dataset Wine
    wine_data = load_wine()
    print("Descrizione del dataset:")
    print(wine_data.DESCR[:500] + "...")

    features = wine_data.data
    labels = wine_data.target
    feature_names = wine_data.feature_names
    target_names = wine_data.target_names

    print(f"\nShape originale features: {features.shape}")
    print(f"Shape labels: {labels.shape}")
    print(f"Classi: {target_names}")
    print(f"Distribuzione classi: {np.bincount(labels)}")

    # Standardizzazione per le features
    print("\n" + "-" * 50)
    print("STANDARDIZZAZIONE E FEATURE SELECTION")
    print("-" * 50)

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)


    # Selezione delle 5 features più significative usando f_classif
    selector = SelectKBest(score_func=f_classif, k=5)
    features_top5 = selector.fit_transform(features_scaled, labels)

    n_components = 5  # Numero fisso di features selezionate


    # Normalizzazione delle componenti per i circuiti quantistici
    normalizer = MinMaxScaler()
    features_top5_normalized = normalizer.fit_transform(features_top5)

    # Split train/test
    train_features, test_features, train_labels, test_labels = train_test_split(
        features_top5_normalized, labels, train_size=0.8, random_state=123, stratify=labels
    )

    print(f"\nTrain set: {len(train_features)} samples")
    print(f"Test set: {len(test_features)} samples")
    print(f"Train class distribution: {np.bincount(train_labels)}")
    print(f"Test class distribution: {np.bincount(test_labels)}")

    # =============================================================================
    # 2. ESECUZIONE ESPERIMENTI QUANTISTICI
    # =============================================================================

    # Numero di qubit basato sulle componenti PCA/kbest scelte
    n_qubits = n_components

    print(f"\n" + "=" * 80)
    print("AVVIO ESPERIMENTI QUANTISTICI")
    print("=" * 80)
    print(f"Iniziando esperimenti con {n_qubits} qubit")
    print(f"Dataset: {len(train_features)} train samples, {len(test_features)} test samples")
    print(f"Classi da predire: {len(target_names)} ({', '.join(target_names)})")

    # Esecuzione di tutti gli esperimenti quantistici
    quantum_results = run_comparative_experiments(
        train_features, train_labels, test_features, test_labels,
        n_qubits, target_names
    )

    # =============================================================================
    # 3. ANALISI COMPLETA DEI RISULTATI
    # =============================================================================

    print("\n" + "=" * 80)
    print("ANALISI FINALE E CONFRONTO")
    print("=" * 80)

    # Analisi dei risultati quantistici
    analyze_results(quantum_results, n_components)

    print("\n" + "=" * 80)
    print("ESPERIMENTI COMPLETATI CON SUCCESSO!")
    print("=" * 80)

    return quantum_results


Questa versione alternativa della funzione main implementa una pipeline completa per la classificazione, basata su una riduzione dimensionale del dataset Wine mediante PCA, al fine di selezionare il numero minimo di feature (e quindi di qubit) necessari a spiegare almeno il 90% della varianza, o alternativamente, cambiando una riga di codice come indicato, a selezionare solo 5 componenti per esprimere le features. L’obiettivo è ridurre lo spazio di input e, di conseguenza, il numero di qubit richiesti nel circuito quantistico, mantenendo al contempo un buon livello di informazione per la classificazione.

In [ ]:
'''
    # Applicazione PCA per spiegare 90% della varianza
    pca = PCA(n_components=0.90)  # 90% della varianza,
     
    #per utilizzare la tecnica di pca con 5 componenti fissate inserire
    #pca=PCA(n_components=5)
    
    
    features_pca = pca.fit_transform(features_scaled)

    n_components = features_pca.shape[1]
'''

Abbiamo definito le varie componenti dei nostri circuiti, che verranno fatte variare per ottenere diverse combinazioni.

Questa funzione ha il compito di generare una collezione dei diversi tipi di feature encoding, utilizzati per mappare dati classici nello spazio di Hilbert dei qubit. L’__input__ richiesto è il numero di qubit (n_qubits), corrispondente al numero di feature selezionate nel preprocessing dei dati. L’__output__ è un dizionario contenente tre diverse strategie di codifica implementate tramite classi della libreria Qiskit Machine Learning.

I tipi di encoding scelti per i nostri confronti sono:

1. ZFeatureMap: codifica le feature attraverso rotazioni attorno all'asse Z, senza entanglement tra i qubit.
2. ZZFeatureMap: introduce interazioni di tipo ZZ tra coppie di qubit, permettendo l’introduzione di correlazioni non lineari.
3. PauliFeatureMap: codifica più generale che combina rotazioni secondo operatori di Pauli scelti (di default X, Y, Z), con possibilità di entanglement controllato.

Questi circuiti sono impiegati successivamente nella costruzione di modelli di classificazione quantistica, svolgendo un ruolo cruciale nella trasformazione dei dati classici in __input__ compatibili con la computazione quantistica.

In [ ]:
def create_encoding_circuits(n_qubits):
    """Crea i diversi tipi di encoding"""
    encodings = {}

    # ZFeatureMap
    encodings['ZFeatureMap'] = ZFeatureMap(feature_dimension=n_qubits, reps=1)

    # ZZFeatureMap
    encodings['ZZFeatureMap'] = ZZFeatureMap(feature_dimension=n_qubits, reps=1)

    # PauliFeatureMap
    encodings['PauliFeatureMap'] = PauliFeatureMap(feature_dimension=n_qubits, paulis=['XX', 'YY', 'ZZ'], reps=1)

    return encodings

La funzione create_ansatz_circuits si occupa della costruzione dei diversi ansatz quantistici, ovvero dei circuiti parametrizzati utilizzati come parte variabile dell'algoritmo VQC (Variational Quantum Classifier). Un ansatz definisce la struttura del circuito quantistico su cui l’ottimizzatore classico agisce per apprendere una funzione decisionale.

In particolare, la funzione accetta in input il numero di qubit (determinato dal numero di feature in input) e restituisce un dizionario contenente tre diverse tipologie di ansatz:

- RealAmplitudes: un ansatz semplice, che alterna rotazioni Ry e porte CNOT, controllato dal parametro reps che definisce la profondità del circuito.

- TwoLocal: un ansatz flessibile che consente di scegliere tipi di rotazioni e interazioni tra qubit; in questo caso è configurato con rotazioni Ry e accoppiamenti CX in uno schema reverse_linear.

- EfficientSU2: un ansatz ispirato a strutture di reti neurali, con un buon compromesso tra espressività e profondità, spesso utilizzato in applicazioni di machine learning quantistico.

Questi circuiti vengono successivamente testati e confrontati per valutarne l’efficacia nella classificazione all’interno della pipeline sperimentale.

In [ ]:
def create_ansatz_circuits(n_qubits):
    """Crea i diversi tipi di ansatz"""
    ansatz_dict = {}

    # RealAmplitudes
    ansatz_dict['RealAmplitudes'] = RealAmplitudes(n_qubits,entanglement='circular', reps=2)

    # TwoLocal
    ansatz_dict['TwoLocal'] = TwoLocal(n_qubits, 'ry', 'cx', 'reverse_linear', reps=2)

    # EfficientSU2 (Neural Network-like)
    ansatz_dict['EfficientSU2'] = EfficientSU2(n_qubits, reps=1)

    return ansatz_dict


La classe QuantumWineClassifier incapsula l’intero processo di codifica, costruzione e valutazione di circuiti quantistici ibridi per la classificazione multiclasse sul nostro dataset.
# Inizializzazione del modello
Il costruttore __init__ riceve in input due circuiti quantistici:

* un feature map che codifica i dati classici in stati quantistici,

* un ansatz variazionale che introduce parametri ottimizzabili nel circuito.

Tali circuiti vengono concatenati mediante compose e completati da un’operazione di misura su tutti i qubit (measure_all). Vengono inoltre inizializzate una lista per la storia dei costi e una variabile per i parametri ottimali, utilizzati per ottenere i plots finali.

# Metodi principali
* circuit_instance: genera un'istanza concreta del circuito, assegnando:
    1. i valori delle feature del singolo campione al feature map,
    2. i parametri variazionali all’ansatz.

* interpreter: decodifica una stringa di bit misurata dal circuito in una classe predetta (0, 1 o 2), usando il peso di Hamming modulo 3. Questa funzione rappresenta un meccanismo di decisione non lineare basato sull’output quantistico.

* label_probability: converte un dizionario di conteggi (bitstring -> count) in una distribuzione di probabilità sulle classi, sommando le frequenze relative dei bitstring associati ad ogni classe.

* classification_probability: calcola, per un insieme di dati e parametri variazionali, la probabilità di appartenenza a ciascuna classe. Per ciascun punto dati, costruisce un circuito parametrizzato, lo esegue con un simulatore a stato vettoriale (StatevectorSampler) e ne aggrega i risultati.

* cost_function: valuta la qualità del modello sui dati forniti tramite la cross-entropy loss tra etichette reali e probabilità predette. Registra inoltre la storia dei valori della funzione obiettivo, utile per l’analisi dell’ottimizzazione.

# Finalità
La classe è concepita per essere utilizzata all’interno di un ciclo di addestramento variazionale (VQC), in cui i parametri dell’ansatz vengono ottimizzati per minimizzare la funzione di costo. Essa costituisce il nucleo operativo del classificatore quantistico sviluppato nel progetto, ed è parte integrante del confronto sperimentale con i metodi classici.

In [ ]:
class QuantumWineClassifier:
    def __init__(self, feature_map, ansatz):
        self.feature_map = feature_map
        self.ansatz = ansatz
        self.circuit = feature_map.compose(ansatz)
        self.circuit.measure_all() #aggiungiamo delle misure per usare il nostro sampler quando valuteremo il circuito
        self.history = []
        self.optimal_params = None
        
    def circuit_instance(self, data_point, variational_params):
        """Crea un'istanza del circuito con parametri specifici"""
        parameters = {}

        # Parametri del feature map
        for i, p in enumerate(self.feature_map.ordered_parameters):
            parameters[p] = data_point[i]

        # Parametri dell'ansatz
        for i, p in enumerate(self.ansatz.ordered_parameters):
            parameters[p] = variational_params[i]

        return self.circuit.assign_parameters(parameters)

    def interpreter(self, bitstring):
        """Interpreta la stringa di bit come classe"""
        hamming_weight = sum(int(k) for k in list(bitstring))
        return (hamming_weight) % 3

    def label_probability(self, results):
        """Calcola le probabilità per ogni classe"""
        shots = sum(results.values())
        probabilities = {0: 0, 1: 0, 2: 0}

        for bitstring, counts in results.items():
            label = self.interpreter(bitstring)
            probabilities[label] += counts / shots

        return probabilities

    def classification_probability(self, data, variational_params):
        """Calcola le probabilità di classificazione per un batch di dati"""
        circuits = [self.circuit_instance(point, variational_params) for point in data]
        sampler = StatevectorSampler() 
        #eseguo per numero di shots di default 1024, ma si può cambiare
        results = sampler.run(circuits).result() #prende risultato in maniera asincrona con result, 
                                                 #contiene i dati associatia  a ogni possibile misuraizoni del registro classico
                                                 #che con measure_all si chiama meas

        classifications = []
        for i, circuit in enumerate(circuits):
            probs = self.label_probability(results[i].data.meas.get_counts()) #da quel registro classico meas, prendo i 
                                                                              #dati della result, da questo estraggo i risultati di meas
                                                                              #da questo mi prendo i counts, cioe num di volte in cui è stato misurato lo stato base
                                                                              #es {'110': 514, '101': 510} sul numero di shots abbiamo misurato lo stato 101 501 volte 
                                                                              #e quello 110 514
            classifications.append(probs)

        return classifications

    def cost_function(self, variational_params, data, labels):
        """Funzione di costo (cross-entropy loss)"""
        classifications = self.classification_probability(data, variational_params)
        y_pred = [[p[0], p[1], p[2]] for p in classifications]
        cost = log_loss(y_true=labels, y_pred=y_pred) #la log_loss importata è quella di sklearn che calcola la cross-entropy loss
        self.history.append(cost)
        
        return cost

La funzione train si occupa della fase di addestramento del classificatore quantistico, ovvero dell’ottimizzazione dei parametri dell’ansatz al fine di minimizzare la cross-entropy.

Il metodo accetta come input:

- train_data e train_labels: rispettivamente i dati e le etichette del training set (già preprocessati e codificati per l’input quantistico),

- optimizer: il metodo di ottimizzazione classico da utilizzare (tra quelli disponibili con scipy.optimize.minimize),

- max_iter: il numero massimo di iterazioni consentite per l’ottimizzazione.

L’ottimizzazione parte da un insieme casuale di parametri iniziali e utilizza una funzione obiettivo (la cost_function) che misura l’errore del modello. Tra gli ottimizzatori utilizzati vi sono:

- COBYLA

- L-BFGS-B

- SLSQP

- POWELL

- NELDER-MEAD

Durante l’addestramento vengono registrati il tempo impiegato, i parametri ottimali trovati, il valore finale della funzione di costo, il numero di iterazioni effettuate e un flag che indica se l’ottimizzazione è andata a buon fine. Questi risultati sono poi restituiti in forma di dizionario, utili per l’analisi comparativa tra diverse configurazioni di esperimenti.



In [ ]:
class QuantumWineClassifier(QuantumWineClassifier):    
    def train(self, train_data, train_labels, optimizer='COBYLA', max_iter=5000):
        """Addestra il classificatore quantistico"""
        self.history = []  #storico delle valutazioni della funzione di costo
        initial_params = np.random.uniform(0, 2*np.pi, self.ansatz.num_parameters) #inizializza i parametri in un range da 0 a 2*pi 

        objective = lambda params: self.cost_function(params, train_data, train_labels)

        start_time = time.time()

        if optimizer == 'COBYLA':#AGGIUNGERE EVETUALI ALTRI OTTIMIZZATORI 
            result = minimize(objective, initial_params, method='COBYLA',
                            options={'maxiter': max_iter})
        elif optimizer == 'L_BFGS_B':
            result = minimize(objective, initial_params, method='L-BFGS-B',
                            options={'maxiter': max_iter})
        elif optimizer == 'SLSQP':
            result = minimize(objective, initial_params, method='SLSQP',
                            options={'maxiter': max_iter})
        elif optimizer == 'POWELL':
            result = minimize(objective, initial_params, method='Powell',
                            options={'maxiter': max_iter})
        elif optimizer == 'NELDER-MEAD':
            result = minimize(objective, initial_params, method='Nelder-Mead',
                            options={'maxiter': max_iter})
        else:
            raise ValueError(f"Optimizer {optimizer} non supportato")

        training_time = time.time() - start_time

        self.optimal_params = result.x
        final_cost = result.fun

        return {
            'optimal_params': self.optimal_params,
            'final_cost': final_cost,
            'training_time': training_time,
            'iterations': len(self.history),
            'success': result.success
        }

Questi due metodi forniscono le funzionalità operative per effettuare predizioni su nuovi dati e per valutare le prestazioni del modello quantistico addestrato, una volta ottenuti i parametri ottimali dell’ansatz.

__predict(data)__
Questo metodo esegue la predizione delle classi per un insieme di dati di input (data), utilizzando i parametri ottimali ottenuti durante la fase di addestramento. In dettaglio:
* Verifica preliminarmente che il modello sia stato addestrato (ovvero che self.optimal_params non sia None);

* Chiama il metodo classification_probability, che restituisce, per ogni campione, una distribuzione di probabilità sulle tre classi;

* Estrae per ciascun punto dati la classe con probabilità massima, producendo una lista di classi predette.

Restituisce una tupla contenente:

* le classi predette,
* le relative distribuzioni di probabilità.

__evaluate(data, labels)__
Il metodo evaluate permette di valutare le prestazioni predittive del modello quantistico su un insieme di test. In particolare:

* Esegue le predizioni richiamando predict;

* Confronta le etichette predette con quelle reali (labels) tramite la metrica di accuracy, ovvero la frazione di classificazioni corrette.

Il metodo restituisce:

* l’accuratezza complessiva del modello sui dati di test,

* l’elenco delle classi predette.


In [ ]:
class QuantumWineClassifier(QuantumWineClassifier):
    def predict(self, data):
        """Predizione usando i parametri ottimali"""
        if self.optimal_params is None:
            raise ValueError("Modello non addestrato")

        probabilities = self.classification_probability(data, self.optimal_params)
        predictions = [max(p, key=p.get) for p in probabilities]
        return predictions, probabilities

    def evaluate(self, data, labels):
        """Valuta le performance del modello"""
        predictions, _ = self.predict(data)
        accuracy = accuracy_score(labels, predictions)
        return accuracy, predictions


La funzione run_comparative_experiments costituisce il cuore della fase sperimentale del progetto. Il suo obiettivo è testare sistematicamente tutte le combinazioni possibili di encoding, ansatz e ottimizzatori all’interno della pipeline di classificazione quantistica. Essa riceve in input i dati di addestramento e test, il numero di qubit (determinato inizialmente), e i nomi delle classi.

All'interno della funzione:

- Vengono inizializzati i diversi circuiti di encoding (feature map), gli ansatz parametrizzati e gli ottimizzatori classici scelti per la fase di training.

- Per ogni combinazione encoding–ansatz–ottimizzatore, viene creato un oggetto QuantumWineClassifier e addestrato sul training set.

- Dopo l'addestramento, il classificatore viene valutato sia sul training set che sul test set, e vengono calcolate le principali metriche di classificazione (accuratezza, precisione, recall e F1-score).

- Per ciascun esperimento riuscito, viene generata e visualizzata una confusion matrix ed un grafico di convergenza della funzione di costo durante l’ottimizzazione.

Ogni esperimento viene salvato sotto forma di dizionario, che include non solo le metriche ma anche informazioni sul numero di parametri, il tempo di training, il numero di qubit e l’esito dell’esperimento. Al termine dell’esecuzione, la funzione restituisce una lista completa dei risultati, utile per successive analisi comparative.

Questo approccio esaustivo consente di valutare in modo approfondito l’impatto delle diverse scelte progettuali (encoding, ansatz, ottimizzazione) sulle prestazioni del classificatore quantistico, fornendo una base quantitativa solida per confronti con i modelli classici e per identificare le configurazioni più promettenti.

In [ ]:
def run_comparative_experiments(train_features, train_labels, test_features, test_labels, n_qubits, target_names):
    """Esegue tutti gli esperimenti comparativi con metriche dettagliate"""

    print("\n" + "=" * 80)
    print("ESPERIMENTI COMPARATIVI VQC CON METRICHE DETTAGLIATE")
    print("=" * 80)

    # Creazione dei componenti
    encodings = create_encoding_circuits(n_qubits)
    ansatz_circuits = create_ansatz_circuits(n_qubits)
    optimizers = ['COBYLA', 'L_BFGS_B', 'SLSQP','POWELL','NELDER-MEAD']  

    results = []
    experiment_count = 0
    total_experiments = len(encodings) * len(ansatz_circuits) * len(optimizers)

    print(f"Totale esperimenti da eseguire: {total_experiments}")
    print(f"Numero di qubit per esperimento: {n_qubits}")
    print("-" * 80)

    for enc_name, encoding in encodings.items():
        for ans_name, ansatz in ansatz_circuits.items():
            for optimizer in optimizers:
                experiment_count += 1
                print(f"\n{'='*20} ESPERIMENTO {experiment_count}/{total_experiments} {'='*20}")
                print(f"Encoding: {enc_name}")
                print(f"Ansatz: {ans_name}")
                print(f"Optimizer: {optimizer}")
                print(f"Parametri Ansatz: {ansatz.num_parameters}")

                # Creazione del classificatore
                classifier = QuantumWineClassifier(encoding, ansatz)

                # Addestramento
                print("Addestramento in corso...")
                train_results = classifier.train(
                    train_features, train_labels,
                    optimizer=optimizer, max_iter=5000
                )

                # Valutazione
                train_acc, train_pred = classifier.evaluate(train_features, train_labels)
                test_acc, test_pred = classifier.evaluate(test_features, test_labels)

                # Calcolo delle metriche dettagliate
                test_precision = precision_score(test_labels, test_pred, average='weighted')
                test_recall = recall_score(test_labels, test_pred, average='weighted')
                test_f1 = f1_score(test_labels, test_pred, average='weighted')

                print(f"RISULTATI:")
                print(f" Train Accuracy: {train_acc:.4f}")
                print(f" Test Accuracy:  {test_acc:.4f}")
                print(f" Test Precision: {test_precision:.4f}")
                print(f" Test Recall:    {test_recall:.4f}")
                print(f" Test F1-Score:  {test_f1:.4f}")
                print(f" Final Cost:     {train_results['final_cost']:.4f}")
                print(f" Training Time:  {train_results['training_time']:.2f}s")

                # Confusion Matrix per questo esperimento
                experiment_title = f"{enc_name} + {ans_name} + {optimizer}"
                print(f"\n CONFUSION MATRIX - {experiment_title}")

                cm = confusion_matrix(test_labels, test_pred)
                plt.figure(figsize=(8, 6))
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                            xticklabels=target_names, yticklabels=target_names)
                plt.title(f'Confusion Matrix\n{experiment_title}')
                plt.xlabel('Predicted')
                plt.ylabel('True')
                plt.show()

                # Grafico della convergenza
                if len(classifier.history) > 1:
                    plt.figure(figsize=(10, 6))
                    plt.plot(classifier.history, 'b-', linewidth=2)
                    plt.title(f'Training Convergence - {experiment_title}')
                    plt.xlabel('Iteration')
                    plt.ylabel('Cost (Log Loss)')
                    plt.grid(True, alpha=0.3)
                    plt.show()

                # Salvataggio risultati
                result = {
                    'encoding': enc_name,
                    'ansatz': ans_name,
                    'optimizer': optimizer,
                    'train_accuracy': train_acc,
                    'test_accuracy': test_acc,
                    'test_precision': test_precision,
                    'test_recall': test_recall,
                    'test_f1': test_f1,
                    'final_cost': train_results['final_cost'],
                    'training_time': train_results['training_time'],
                    'iterations': train_results['iterations'],
                    'success': train_results['success'],
                    'num_parameters': ansatz.num_parameters,
                    'num_qubits': n_qubits
                }

                results.append(result)

            print("-" * 80)

    return results

La funzione analyze_results ha lo scopo di sintetizzare e visualizzare i risultati degli esperimenti quantistici, al fine di effettuare un confronto sistematico tra le diverse configurazioni testate. Essa prende in input:

- l’elenco dei risultati degli esperimenti (results),

- il numero di qubit utilizzati.

I risultati vengono prima convertiti in un DataFrame pandas, facilitando l’analisi statistica. Vengono quindi calcolate statistiche aggregate sui soli esperimenti che hanno avuto successo (success == True), tra cui:

- accuratezza media su training e test set,

- precisione, richiamo e F1-score medi,

- tempo medio di addestramento.

Viene inoltre fornito un ranking delle 10 migliori configurazioni quantistiche in base alla test accuracy, visualizzando anche gli encoding utilizzati, l’ansatz, l’ottimizzatore e le metriche ottenute. Questo permette di identificare facilmente le combinazioni più promettenti e confrontarle.

In [ ]:
def analyze_results(results, n_components):
    """Analizza e visualizza i risultati degli esperimenti con grafici avanzati"""

    print("\n" + "=" * 80)
    print("ANALISI AVANZATA DEI RISULTATI")
    print("=" * 80)

    # Conversione in DataFrame
    df_results = pd.DataFrame(results)

    # Statistiche generali
    print("\n1. STATISTICHE GENERALI")
    print("-" * 40)
    successful_experiments = df_results[df_results['success'] == True]
    print(f"Esperimenti riusciti: {len(successful_experiments)}/{len(df_results)}")
    print(f"Numero di qubit utilizzati: {n_components}")
    print(f"Accuracy media quantistica (train): {successful_experiments['train_accuracy'].mean():.4f} ± {successful_experiments['train_accuracy'].std():.4f}")
    print(f"Accuracy media quantistica (test): {successful_experiments['test_accuracy'].mean():.4f} ± {successful_experiments['test_accuracy'].std():.4f}")
    print(f"Precision media (test): {successful_experiments['test_precision'].mean():.4f} ± {successful_experiments['test_precision'].std():.4f}")
    print(f"Recall media (test): {successful_experiments['test_recall'].mean():.4f} ± {successful_experiments['test_recall'].std():.4f}")
    print(f"F1-Score media (test): {successful_experiments['test_f1'].mean():.4f} ± {successful_experiments['test_f1'].std():.4f}")
    print(f"Tempo medio di training: {successful_experiments['training_time'].mean():.2f}s ± {successful_experiments['training_time'].std():.2f}s")

    # Top 10 configurazioni per test accuracy
    print("\n2. TOP 10 CONFIGURAZIONI (Test Accuracy)")
    print("-" * 80)
    top_configs = successful_experiments.nlargest(10, 'test_accuracy')
    print(f"{'Rank':<4} {'Encoding':<15} {'Ansatz':<15} {'Optimizer':<8} {'Test Acc':<8} {'Precision':<9} {'Recall':<7} {'F1':<7} {'Time(s)':<7}")
    print("-" * 80)
    for rank, (idx, row) in enumerate(top_configs.iterrows(), 1):
        print(f"{rank:<4} {row['encoding']:<15} {row['ansatz']:<15} {row['optimizer']:<8} "
              f"{row['test_accuracy']:<8.4f} {row['test_precision']:<9.4f} {row['test_recall']:<7.4f} "
              f"{row['test_f1']:<7.4f} {row['training_time']:<7.1f}")

In [ ]:
# Esecuzione del main
if __name__ == "__main__":
    quantum_results, classic_metrics = main()